In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import ipywidgets #as widgets
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
high_res_time_series_df = pd.read_csv(
    "Corrected_power_time_series_201_steps.csv",
    on_bad_lines = 'warn')

In [ ]:
high_res_time_series_df.set_index('Parent Isotope', inplace = True)
time_array = [float(t) for t in high_res_time_series_df.columns]

In [ ]:
def get_max(entries):
  cols = [''.join([l for l in entry if l.isdigit()]) + 
          ''.join([l for l in entry if l.isalpha()])
          for entry in entries.split(",")]
  max_y = np.max(high_res_time_series_df.loc[cols[0]])
  for col in cols[1:]:
    max_y = max(max_y, np.max(high_res_time_series_df.loc[col]))
  return max_y
def plot(entries):
  fig = go.Figure()
  if "," in entries:
    title = "Power Produced by 1 Gram Each of "
  else:
    title = "Power Produced by 1 Gram of "
  for entry in entries.split(","):
    A = int(''.join([l for l in entry if l.isdigit()]))
    El = ''.join([l for l in entry if l.isalpha()])
    entry = str(A) + El
    if np.max(high_res_time_series_df.loc[entry]) < 10**-6 :
      fig.update_layout(title = entry + " produces less than 1μW") 
      return fig
    else:
      fig.add_trace(go.Scatter(x = time_array,
            y = high_res_time_series_df.loc[entry] / A,
            name = El + "-" + str(A), mode = "markers"))
      y_dict = dict(type = "log", title = "Power (Watts)", nticks = 16,
              range = [-6, np.log10(get_max(entries))] )
      x_dict = dict(type = "log", title = "Time (Seconds)", nticks = 24,
                    range = [-10, 10])
      fig.update_layout(xaxis = x_dict, yaxis = y_dict,
              title = title + entries)  
  return fig

In [ ]:
from IPython.display import display, clear_output
button = ipywidgets.Button(description='Make Plot')
button_multiple = ipywidgets.Button(description='Make Plot')
output = ipywidgets.Output()
dropdown = ipywidgets.Dropdown(
            options=[x for x in high_res_time_series_df.index],
            value = "3H")
text_box = ipywidgets.Text(description="Type here")

def on_button_clicked_1_isotope(b):
  with output:
    clear_output()
    plot(dropdown.value).show()

button.on_click(on_button_clicked_1_isotope)

def on_button_clicked_multiple_isotopes(c):
  with output:
    clear_output()
    plot(text_box.value).show()   

button_multiple.on_click(on_button_clicked_multiple_isotopes)

#Plot Data Interactively
After clicking on the dropdown box, you can type the name of an isotope to find it in the list. Click "Make Plot" once you have selected an isotope. 

In [ ]:
display(dropdown, button, output)

#Plot and compare multiple isotopes
Enter the isotope(s) to plot, each isotope separated by a comma. Click "Make Plot" once you have entered the isotope(s). 

In [ ]:
display(text_box, button_multiple, output)